In [1]:
import numpy as np
from gensim.models import KeyedVectors
import pandas as pd 
from keras.models import Sequential 
from keras.layers import Convolution1D,MaxPool1D,GlobalAveragePooling1D
from keras.layers import Input

In [2]:
#获取描述矩阵
def getToken(dirin,max_len,embedding_dim,wv_from_text):
    data_t=pd.read_csv(dirin,names=['id','entity','token']) #获取分词描述
    data_matrix_raw=list() 
    for i in range(len(data_t)): 
        description_raw=str(data_t['token'][i]).split()
        if len(description_raw)>375:
            print(i)
        description=description_raw[:373]

        des_matrix=list()
        for j in range(len(description)):
            word=description[j]
            if word in  wv_from_text:#如果存在词，获取词向量
                des_matrix.append(wv_from_text.get_vector(word))
            else:#如果不存在词，设置为零向量填充
                des_matrix.append(np.zeros(embedding_dim))
        
        emb=np.array(des_matrix) 
        emb=np.pad(emb,((0,max_len-len(emb)),(0,0)),'constant') #描述矩阵填充为375*100的矩阵,填充值为0
        data_matrix_raw.append(emb)
        #data_matrix_raw.append(np.zeros((max_len, embedding_dim)))#记录某个描述的初始嵌入矩阵 
    data_matrix=np.array(data_matrix_raw)#获取每个实体的词嵌入矩阵
    return data_matrix

In [3]:
#搭建模型，两层CNN,对比论文使用的参数
def getCNNEmb(data_matrix): 
    model = Sequential() 
    model.add(Input(shape=(max_len, embedding_dim)))  # 输入特征接收维度)  # 词嵌入层  
    # 1D 卷积层，对词嵌入层输出做卷积操作
    model.add(Convolution1D(kernel_size=2,filters=100,activation='relu'))
    model.add(MaxPool1D(pool_size=20)) 
    model.add(Convolution1D(kernel_size=2,filters=100,activation='relu'))
    model.add(GlobalAveragePooling1D())  
    #model.summary()
    #返回隐层特征
    out=model(data_matrix)
    return out.numpy()

In [4]:
max_len=375#最长实体描述长度，设为375和对比论文一样。
embedding_dim=100 #词向量维度 

In [5]:
#加载词嵌入模型
wv_from_text = KeyedVectors.load_word2vec_format("preModel/w2v_vec.txt", binary=False)   #从文本加载词向量，加载预训练的word2vec
vocab_size = len(wv_from_text) + 1    # 加1是因为无单词则索引为0
#w2index=wv_from_text.key_to_index  #获取词的索引
#通过embedding_vector = wv_from_text.get_vector(word)  # 获取对应的词向量

In [6]:
entity_matrix=getToken('../split/entity_tokens_nltk.csv',max_len,embedding_dim,wv_from_text)#获取每个实体的词嵌入矩阵
print(entity_matrix.shape,type(entity_matrix))
entity_vector=getCNNEmb(entity_matrix)#获得word2vec+CNN的语义特征
print(entity_vector.shape,type(entity_vector))
np.save('preEmb/entity_des_cnn_100.npy',entity_vector)

(7199, 375, 100) <class 'numpy.ndarray'>
(7199, 100) <class 'numpy.ndarray'>


In [7]:
relation_matrix=getToken('../split/relation_tokens_nltk.csv',max_len,embedding_dim,wv_from_text)#获取每个实体的词嵌入矩阵
print(relation_matrix.shape,type(relation_matrix))
relation_vector=getCNNEmb(relation_matrix)#获得word2vec+CNN的语义特征
print(relation_vector.shape,type(relation_vector))
np.save('preEmb/relation_des_cnn_100.npy',relation_vector)

(15, 375, 100) <class 'numpy.ndarray'>
(15, 100) <class 'numpy.ndarray'>
